In [ ]:
# here only executting stuff. no functions or classes

In [ ]:
## um die models und functions hier nutzen zu können, muss man die datein als module importieren. 
## das geht auch wenn das notebeooks sind, ist aber deutlich einfacher, wenn es einfach .py dateien sind.

In [ ]:
!pip install -r requirements.txt

## CycleGAN Maps 


In [7]:
#!/usr/bin/env python3
#-*- coding:utf-8 -*-
#$ -l cuda=1 # remove this line when no GPU is needed!
#$ -q all.q # do not fill the qlogin queue
#$ -cwd # start processes in current working directory
#$ -V # provide environment variables to processes
#$ -t 1-8 # start 8 instances: to train different models in parallel
#Cluster settings, 

try:
    model_param_id = int(os.environ['SGE_TASK_ID'])
except:
    print("no SGE_TASK_ID set, choosing default model parameters ")
    model_param_id = 0 #param_train_cycle_list[0] should be default model params

import importlib
import models
import functions 
import torch
import torch.nn as nn
importlib.reload(models)
importlib.reload(functions)
import numpy as np
#different import way for cluster
'''
import imp
models = imp.load_source('models', './models.py')
functions = imp.load_source('functions', './functions.py')
'''

# init CycleGAN
genA2B = models.Generator(input_nc=3, output_nc=3, n_residual_blocks=9)
genB2A = models.Generator(input_nc=3, output_nc=3, n_residual_blocks=9)
discA  = models.Discriminator(input_nc=3)
discB  = models.Discriminator(input_nc=3)
classifier = models.Classifier().net
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
databaseName = "maps"
root_path_data = "./data/"+databaseName
root_path_checkpoints = "./checkpoints/"+databaseName

targetEpoch = 2
saveAt = 1 #how often to store the model at training
print("the device ist at")
print(device)

if device == torch.device("cuda"):
    #debug
    print("should load models to cuda")
    print("device == cuda -> "+ str(device == "cuda:0"))
    #model.load_state_dict(torch.load("{}/{} epoch{}".format(path, name, epoch)))
else:
    #debug
    print("should load models to cpu")
    print("device == cuda -> "+ str(device == "cuda:0"))
    #model.load_state_dict(torch.load("{}/{} epoch{}".format(path, name, epoch),map_location=torch.device('cpu')))

cycle  = models.CycleGAN(genA2B, genB2A, discA, discB, classifier, device, root_path_data, root_path_checkpoints)


param_train1  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advMSE_l10", resnet_blocks=9, loss_adv=torch.nn.MSELoss(), lambdas=(10,0.5)) # default
param_train2  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advMSE_l5" , resnet_blocks=9, loss_adv=torch.nn.MSELoss(), lambdas=(5,0.5))
param_train3  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advL1_l10", resnet_blocks=9, loss_adv=torch.nn.L1Loss(), lambdas=(10,0.5))
param_train4  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advL1_l5" , resnet_blocks=9, loss_adv=torch.nn.L1Loss(), lambdas=(5,0.5))
param_train5  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advMSE_l10", resnet_blocks=5, loss_adv=torch.nn.MSELoss(), lambdas=(10,0.5))
param_train6  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advMSE_l5" , resnet_blocks=5, loss_adv=torch.nn.MSELoss(), lambdas=(5,0.5))
param_train7  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advL1_l10", resnet_blocks=5, loss_adv=torch.nn.L1Loss(), lambdas=(10,0.5))
param_train8  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advL1_l5" , resnet_blocks=5, loss_adv=torch.nn.L1Loss(), lambdas=(5,0.5))
param_train_cycle_list = [param_train1, param_train2, param_train3, param_train4, param_train5, param_train6, param_train7, param_train8]

# paramter to load the background error verison. since this extension needed major changes in training/loaded training was done locally.
param_back_err = models.Param(name ="background_default", epochs = 80)

# paramters needed to call eval_testset() function
param_eval_testset = models.Param(channels = 3, size= 256) 

# parameters used for training the classifier
param_train_classifier = models.Param(channels = 3, epochs = targetEpoch, size = 256, name = "classifier_test")

no SGE_TASK_ID set, choosing default model parameters 
the device ist at
cuda
should load models to cuda
device == cuda -> False


## Download Dataset

In [45]:
functions.get_data_h2z(databaseName)

creating zebra folder and downloading data
...2520%, 0 MB, 34414 KB/s, 0 seconds passedunzipping files

finished download


## Train Model

In [43]:
param_train_cycle = param_train_cycle_list[model_param_id-1]
cycle.train(param_train_cycle)   

training started!...
logfile running at ./checkpoints/cityscapes/logcycle_r5_advL1_l5.txt


FileNotFoundError: [Errno 2] No such file or directory: './data/cityscapes/trainA/'

## Generate Results

In [2]:
#for every model --TODO
#for paramItem in param_train_cycle_list:
for paramItem in [param_train3,param_train8]:
    #load model
    cycle.load_cycle_nets(epoch = 100, model_name = paramItem.name)
    print(device)
    #generate for every test image some output image
    cycle._create_evalset_paired(paramItem,subfolder=paramItem.name)
    #calculate error with expected image

#compare errors between models

#run pix2pix ? and compare it with that?


loading models to cuda
loading models to cuda
loading models to cuda
loading models to cuda
cuda
finished domain A
finished domain B
loading models to cuda
loading models to cuda
loading models to cuda
loading models to cuda
cuda
finished domain A
finished domain B


## Calculate Error for each model

In [8]:
#MSE? SSIM? FCN? 
import os
from PIL import Image
import torchvision.transforms as transforms


losses= {}
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    subfolder = paramItem.name
    losses[paramItem.name]={}
    for source in source_domains:  
        losses[paramItem.name][source]=[]
        folder=root_path_data + "/eval{}/{}/".format(source,subfolder)
        folderSize = int(len(os.listdir(folder)) / 3)
        for i in range(0,folderSize):
            gen="{}{}_generated.jpg".format(folder, i)
            exp="{}{}_expected.jpg".format(folder, i)
            gen=Image.open(gen)
            exp=Image.open(exp)
            loss=functions.calcMSE(gen,exp)
            losses[paramItem.name][source].append(loss)
    

## display losses

In [30]:
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ = np.array(losses[paramItem.name][source]).sum()
        std = np.std(losses[paramItem.name][source])
        overall = np.array(losses[paramItem.name]['A']).sum() + np.array(losses[paramItem.name]['B']).sum()
        print("param:{} \t source:{} \t std:{:.4f} \t sum:{:.2f} \t overall:{:.2f}".format(paramItem.name,source,std,summ,overall))

param:cycle_r9_advMSE_l10 	 source:A 	 std:0.0058 	 sum:5.51 	 overall:46.03
param:cycle_r9_advMSE_l10 	 source:B 	 std:0.0114 	 sum:40.52 	 overall:46.03
param:cycle_r9_advMSE_l5 	 source:A 	 std:0.0055 	 sum:5.39 	 overall:45.65
param:cycle_r9_advMSE_l5 	 source:B 	 std:0.0115 	 sum:40.26 	 overall:45.65
param:cycle_r9_advL1_l10 	 source:A 	 std:0.0054 	 sum:5.35 	 overall:49.64
param:cycle_r9_advL1_l10 	 source:B 	 std:0.0119 	 sum:44.29 	 overall:49.64
param:cycle_r9_advL1_l5 	 source:A 	 std:0.0056 	 sum:5.49 	 overall:49.93
param:cycle_r9_advL1_l5 	 source:B 	 std:0.0120 	 sum:44.44 	 overall:49.93
param:cycle_r5_advMSE_l10 	 source:A 	 std:0.0054 	 sum:5.31 	 overall:45.62
param:cycle_r5_advMSE_l10 	 source:B 	 std:0.0120 	 sum:40.31 	 overall:45.62
param:cycle_r5_advMSE_l5 	 source:A 	 std:0.0054 	 sum:5.40 	 overall:44.33
param:cycle_r5_advMSE_l5 	 source:B 	 std:0.0107 	 sum:38.93 	 overall:44.33
param:cycle_r5_advL1_l10 	 source:A 	 std:0.0055 	 sum:5.53 	 overall:46.46
para

In [5]:
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ = np.array(losses[paramItem.name][source]).sum()
        std = np.std(losses[paramItem.name][source])
        overall = np.array(losses[paramItem.name]['A']).sum() + np.array(losses[paramItem.name]['B']).sum()
        print("param:{} source:{} std:{:2f} sum:{:0f} overall:{}".format(paramItem.name,source,std,summ,overall))

param:cycle_r9_advMSE_l10 source:A std:0.020727 sum:42.331000 overall:200.31400000000002
param:cycle_r9_advMSE_l10 source:B std:0.027450 sum:157.983000 overall:200.31400000000002
param:cycle_r9_advMSE_l5 source:A std:0.019033 sum:41.606000 overall:198.77599999999998
param:cycle_r9_advMSE_l5 source:B std:0.027498 sum:157.170000 overall:198.77599999999998
param:cycle_r9_advL1_l10 source:A std:0.018360 sum:41.249000 overall:205.97
param:cycle_r9_advL1_l10 source:B std:0.027445 sum:164.721000 overall:205.97
param:cycle_r9_advL1_l5 source:A std:0.018212 sum:40.860000 overall:205.512
param:cycle_r9_advL1_l5 source:B std:0.027137 sum:164.652000 overall:205.512
param:cycle_r5_advMSE_l10 source:A std:0.017783 sum:41.123000 overall:198.801
param:cycle_r5_advMSE_l10 source:B std:0.028439 sum:157.678000 overall:198.801
param:cycle_r5_advMSE_l5 source:A std:0.018145 sum:41.852000 overall:196.74
param:cycle_r5_advMSE_l5 source:B std:0.025475 sum:154.888000 overall:196.74
param:cycle_r5_advL1_l10 sou

In [73]:
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ = np.array(losses[paramItem.name][source]).sum()
        std = np.std(losses[paramItem.name][source])
        overall = np.array(losses[paramItem.name]['A']).sum() + np.array(losses[paramItem.name]['B']).sum()
        print("param:{} source:{} std:{:2f} sum:{:0f} overall:{}".format(paramItem.name,source,std,summ,overall))

param:cycle_r9_advMSE_l10 source:A std:0.021128 sum:44.468000 overall:207.642
param:cycle_r9_advMSE_l10 source:B std:0.027676 sum:163.174000 overall:207.642
param:cycle_r9_advMSE_l5 source:A std:0.018349 sum:41.879000 overall:212.426
param:cycle_r9_advMSE_l5 source:B std:0.027998 sum:170.547000 overall:212.426
param:cycle_r9_advL1_l10 source:A std:0.020923 sum:43.669000 overall:201.413
param:cycle_r9_advL1_l10 source:B std:0.026593 sum:157.744000 overall:201.413
param:cycle_r9_advL1_l5 source:A std:0.017752 sum:45.098000 overall:202.21200000000002
param:cycle_r9_advL1_l5 source:B std:0.026540 sum:157.114000 overall:202.21200000000002
param:cycle_r5_advMSE_l10 source:A std:0.023278 sum:45.731000 overall:211.471
param:cycle_r5_advMSE_l10 source:B std:0.028075 sum:165.740000 overall:211.471
param:cycle_r5_advMSE_l5 source:A std:0.021359 sum:44.727000 overall:205.86300000000003
param:cycle_r5_advMSE_l5 source:B std:0.027667 sum:161.136000 overall:205.86300000000003
param:cycle_r5_advL1_l10

In [ ]:
bla
for paramItem in param_train_cycle_list:
    print(type(paramItem.name))
    print(type("hallo"))

In [36]:
#calculate differences between 50 and 100
losses50 = np.load(root_path_data+"/pairedlosses50.npy",allow_pickle=True).item()
losses100 = np.load(root_path_data+"/pairedlosses100.npy",allow_pickle=True).item()
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ50 = np.array(losses50[paramItem.name][source]).sum()
        std50 = np.std(losses50[paramItem.name][source])
        overall50 = np.array(losses50[paramItem.name]['A']).sum() + np.array(losses50[paramItem.name]['B']).sum()
        summ100 = np.array(losses100[paramItem.name][source]).sum()
        std100 = np.std(losses100[paramItem.name][source])
        overall100 = np.array(losses100[paramItem.name]['A']).sum() + np.array(losses100[paramItem.name]['B']).sum()
        diffstd = std100-std50
        diffsumm= summ100-summ50
        diffoverall= overall100-overall50
        print("param:{}\n source:{} \n std:{:2f}({:2f}) \n sum:{:0f}({:0f}) \n overall:{}({:2f})".format(paramItem.name,source,std100,diffstd,summ100,diffsumm,overall100,diffoverall))

param:cycle_r9_advMSE_l10
 source:A 
 std:0.020727(-0.000401) 
 sum:42.331000(-2.137000) 
 overall:200.31400000000002(-7.328000)
param:cycle_r9_advMSE_l10
 source:B 
 std:0.027450(-0.000226) 
 sum:157.983000(-5.191000) 
 overall:200.31400000000002(-7.328000)
param:cycle_r9_advMSE_l5
 source:A 
 std:0.019033(0.000683) 
 sum:41.606000(-0.273000) 
 overall:198.77599999999998(-13.650000)
param:cycle_r9_advMSE_l5
 source:B 
 std:0.027498(-0.000500) 
 sum:157.170000(-13.377000) 
 overall:198.77599999999998(-13.650000)
param:cycle_r9_advL1_l10
 source:A 
 std:0.019033(-0.001891) 
 sum:41.606000(-2.063000) 
 overall:198.77599999999998(-2.637000)
param:cycle_r9_advL1_l10
 source:B 
 std:0.027498(0.000904) 
 sum:157.170000(-0.574000) 
 overall:198.77599999999998(-2.637000)
param:cycle_r9_advL1_l5
 source:A 
 std:0.018212(0.000460) 
 sum:40.860000(-4.238000) 
 overall:205.512(3.300000)
param:cycle_r9_advL1_l5
 source:B 
 std:0.027137(0.000597) 
 sum:164.652000(7.538000) 
 overall:205.512(3.300000

In [29]:
losses50.item()

{'cycle_r9_advMSE_l10': {'A': [0.061,
   0.03,
   0.029,
   0.043,
   0.026,
   0.045,
   0.038,
   0.025,
   0.04,
   0.035,
   0.024,
   0.023,
   0.034,
   0.022,
   0.031,
   0.028,
   0.03,
   0.026,
   0.026,
   0.046,
   0.026,
   0.026,
   0.045,
   0.066,
   0.032,
   0.04,
   0.024,
   0.075,
   0.029,
   0.032,
   0.047,
   0.027,
   0.04,
   0.029,
   0.05,
   0.064,
   0.028,
   0.049,
   0.036,
   0.041,
   0.021,
   0.028,
   0.04,
   0.029,
   0.021,
   0.041,
   0.079,
   0.057,
   0.105,
   0.043,
   0.033,
   0.025,
   0.022,
   0.028,
   0.03,
   0.037,
   0.043,
   0.03,
   0.031,
   0.07,
   0.035,
   0.099,
   0.027,
   0.025,
   0.036,
   0.056,
   0.031,
   0.027,
   0.053,
   0.027,
   0.03,
   0.03,
   0.033,
   0.036,
   0.038,
   0.05,
   0.028,
   0.022,
   0.037,
   0.042,
   0.093,
   0.041,
   0.026,
   0.061,
   0.032,
   0.033,
   0.067,
   0.028,
   0.024,
   0.027,
   0.084,
   0.037,
   0.031,
   0.026,
   0.03,
   0.086,
   0.035,
   0.136,
   0.0

In [19]:
type(losses)

dict

In [40]:
model_param_id

0